In [1]:
!pip install plotly
!pip install geopandas

In [2]:
import geopandas as gpd

import numpy as np
import pandas as pd
import plotly.express as px
import json
#import ogr


In [3]:
with open('Community.geojson') as f:
  dxb_communities_GJson = json.load(f)
# dxb_communities_GJson

In [4]:
dxb_communities_GJson["features"][0].keys()
dxb_communities_GJson["features"][0]['properties']

{'Name': 'kml_2',
 'description': None,
 'CNAME_E': "AL BADA'",
 'CNAME_A': 'البدع',
 'OBJECTID': '148',
 'LABEL_E': "AL BADA'",
 'LABEL_A': 'البدع',
 'C_PREFIX_E': 'Community:',
 'C_PREFIX_A': '',
 'COMMUNITY_': "AL BADA' - 333",
 'COMMUNITY1': 'البدع - 333',
 'DGIS_ID': '0000065',
 'COMM_NUM': '333',
 'NDGIS_ID': '65',
 'SHAPE_AREA': '1984700.5385',
 'SHAPE_LEN': '7946.07017526'}

In [5]:
dxb_communities_pop=pd.read_csv("Dxb_Community_Popn_FinalDF.csv")
dxb_communities_pop.head()

,Unnamed: 0,COMM_NUM,SHAPE_AREA,SHAPE_LEN,lat,lon,Sector,CNAME_E,CNAME_A,Pop2017,Pop2018,Pop2019,Pop2019density,PopGrowth19by18,PopGrowthbinned,PopDensitybinned,Counter
0,0,333,1.984701e+06,7946.070175,25.225314,55.268768,Sector 3,AL BADA,البدع,51467.0,54338.0,56734.0,28585.672699,4.409437,2 < x <= 20,10k < x <= 50k,1
1,1,121,8.707842e+05,5165.842443,25.287165,55.317958,Sector 1,CORNICHE DEIRA,كورنيش ديرة,14.0,14.0,14.0,16.077461,0.000000,-2 < x <=2,10 < x <1000,1
2,2,282,1.269407e+07,14582.351508,25.230697,55.521222,Sector 2,AL KHAWANEEJ TWO,الخوانيج الثانية,2786.0,3865.0,4523.0,356.308151,17.024580,2 < x <= 20,10 < x <1000,1
3,3,353,2.430429e+06,6705.662879,25.175690,55.238309,Sector 3,AL SAFFA FIRST,الصفا الأولى,7540.0,7893.0,8514.0,3503.085889,7.867731,2 < x <= 20,1000 < x <= 10k,1
4,4,736,1.305883e+07,15567.139691,25.024015,55.646574,Sector 7,NAZWAH,نزوه,450.0,504.0,544.0,41.657652,7.936508,2 < x <= 20,10 < x <1000,1


In [6]:
dxb_healthcare_venues_incommunity = pd.read_csv("Healthcare_Venues_in_Communities.csv") #.\Datasets\Community.geojson
dxb_healthcare_venues_incommunity.head()

,Unnamed: 0,Venue ID,Venue,Latitude,Longitude,Venue Category,geometry,COMM_NUM
0,0,4dd3feece4cd6154148c0509,Iranian Hospital (المستشفى الإيراني),25.230393,55.269144,Hospital,POINT (55.26914394990134 25.230393457502),333
1,1,5e2006c3e7117a0008c8038e,Fakih IVF - Dubai,25.221963,55.263826,Hospital,POINT (55.2638264 25.2219634),333
2,2,514705f3e4b0da934ca8ef83,Al Noor Polyclinic,25.229366,55.273964,Hospital,POINT (55.27396376413697 25.22936632193941),333
3,3,4bbc246bafe1b713fc21304b,Belhoul European Hospital,25.237432,55.275028,Hospital,POINT (55.27502782512651 25.23743185102106),333
4,4,4bbc29d93de8c9b654b79aad,Medcare Hospital (مستشفى ميدكير),25.183387,55.242802,Hospital,POINT (55.24280221586729 25.18338701350705),353


In [7]:
communitywise_healthcare_venues = dxb_healthcare_venues_incommunity.groupby('COMM_NUM').count()
communitywise_healthcare_venues['COMM_NUM_TEMP'] = communitywise_healthcare_venues.index
communitywise_healthcare_venues.reset_index(drop=True, inplace = True)
communitywise_healthcare_venues=communitywise_healthcare_venues[['COMM_NUM_TEMP','Venue ID']]
communitywise_healthcare_venues

,COMM_NUM_TEMP,Venue ID
0,122,5
1,127,1
2,129,2
3,133,2
4,214,5
...,...,...
57,891,1
58,911,1
59,917,1
60,931,1


In [13]:
community_pop_hcf = pd.merge(left=dxb_communities_pop, right=communitywise_healthcare_venues, left_on='COMM_NUM', right_on='COMM_NUM_TEMP',how='outer')
community_pop_hcf.drop("COMM_NUM_TEMP", axis=1, inplace=True)
community_pop_hcf.drop(community_pop_hcf.columns[0], axis=1, inplace=True)
community_pop_hcf.rename(columns = {'Venue ID':'HealthcareFacilities'}, inplace = True)

community_pop_hcf.to_csv("community_pop_hcf.csv",index=False)

In [10]:
# For Ref only 
#max_value = dxb_communities['Pop2019density'].max()
#                         color=dxb_communities['Cluster'],
#                            color_discrete_sequence=px.colors.qualitative.Bold,
#                            #color_discrete_sequence=["red", "rgba(0,0,0,0.1)", "blue", "green"],# for specific colors to category
#                            category_orders={"Cluster": ["0", "1", "2", "3"]}, # for ordering the categories for legend
#                            range_color=(0, 6),

In [11]:
# # # ##### Continuous Scale ####

# community_pop_hcf_c = community_pop_hcf
# community_pop_hcf_c['HealthcareFacilities'] = community_pop_hcf_c['HealthcareFacilities'].fillna(0)
# community_pop_hcf_c.loc[community_pop_hcf_c['HealthcareFacilities'] == 21, 'HealthcareFacilities'] = np.nan # Removing Healthcare city
# community_pop_hcf_c

# fig = px.choropleth_mapbox(community_pop_hcf_c, 
#                            geojson=dxb_communities_GJson, 
#                            locations='COMM_NUM',
#                            color=community_pop_hcf_c['HealthcareFacilities'],
#                            color_continuous_scale="Ylgn",
#                            range_color=(0, int(community_pop_hcf_c['HealthcareFacilities'].max())),
#                            featureidkey="properties.COMM_NUM",
#                            hover_name='CNAME_E',
#                            hover_data={'PopDensitybinned': False, 'CNAME_E': False, 'Pop2019density': True, 'HealthcareFacilities':True},
#                            mapbox_style='open-street-map',
#                            zoom=9,
#                            center={'lat': 25, 'lon': 55},
#                            opacity=0.6,
#                            width = 1200,
#                            height=800,
#                           )

# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [12]:
# # Discrete

# community_pop_hcf_d = community_pop_hcf
# community_pop_hcf_d['HealthcareFacilities'] = community_pop_hcf_d['HealthcareFacilities'].fillna(0)
# community_pop_hcf_d['HealthcareFacilities'] = community_pop_hcf_d[['HealthcareFacilities']].astype(int)
# community_pop_hcf_d['HealthcareFacilities'] = community_pop_hcf_d[['HealthcareFacilities']].astype(str)

# community_pop_hcf_d.loc[community_pop_hcf_d['HealthcareFacilities'] == '21', 'HealthcareFacilities'] = 'Healthcare City (21)'
# community_pop_hcf_d.loc[community_pop_hcf_d['HealthcareFacilities'] == '0', 'HealthcareFacilities'] = 'Not Present (0)'
# community_pop_hcf_d.to_csv("checking.csv")

# fig = px.choropleth_mapbox(community_pop_hcf_d, 
#                            geojson=dxb_communities_GJson, 
#                            locations='COMM_NUM',
#                            color=community_pop_hcf_d['HealthcareFacilities'],
#                            #color_discrete_sequence=px.colors.qualitative.Bold,
#                            color_discrete_sequence=["yellow", "pink","brown","darkgrey","teal","fuchsia","blue", "green"],# for specific colors to category
#                            category_orders={"HealthcareFacilities": ["Not Present (0)", "1", "2", "3","4","5","6","Healthcare City (21)"]}, # for ordering the categories for legend
#                            #range_color=(0, int(community_pop_hcf['HealthcareFacilities'].max())),
#                            range_color=(0, 12),
#                            featureidkey="properties.COMM_NUM",
#                            hover_name='CNAME_E',
#                            hover_data={'PopDensitybinned': False, 'CNAME_E': False, 'Pop2019density': True, 'HealthcareFacilities':True},
#                            mapbox_style='open-street-map',
#                            zoom=9,
#                            center={'lat': 25, 'lon': 55},
#                            opacity=0.6,
#                            width = 1200,
#                            height=800,
#                           )



# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()



In [14]:
community_pop_hcf_e=community_pop_hcf
community_pop_hcf_e = community_pop_hcf_e.dropna()
community_pop_hcf_e['Pop Per HealthCF'] = community_pop_hcf_e['Pop2019']/(community_pop_hcf_e['HealthcareFacilities']*1000)
community_pop_hcf_e.drop(community_pop_hcf_e[community_pop_hcf_e['HealthcareFacilities'] == 21].index, inplace = True)
community_pop_hcf_e.to_csv("checking.csv")
community_pop_hcf_e

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,COMM_NUM,SHAPE_AREA,SHAPE_LEN,lat,lon,Sector,CNAME_E,CNAME_A,Pop2017,Pop2018,Pop2019,Pop2019density,PopGrowth19by18,PopGrowthbinned,PopDensitybinned,Counter,HealthcareFacilities,Pop Per HealthCF
0,333,1.984701e+06,7946.070175,25.225314,55.268768,Sector 3,AL BADA,البدع,51467.0,54338.0,56734.0,28585.672699,4.409437,2 < x <= 20,10k < x <= 50k,1,4.0,14.1835
3,353,2.430429e+06,6705.662879,25.175690,55.238309,Sector 3,AL SAFFA FIRST,الصفا الأولى,7540.0,7893.0,8514.0,3503.085889,7.867731,2 < x <= 20,1000 < x <= 10k,1,3.0,2.8380
5,216,4.840781e+06,10171.569888,25.224103,55.391648,Sector 2,AL RASHIDIYA,الراشدية,33719.0,34594.0,37167.0,7677.892532,7.437706,2 < x <= 20,1000 < x <= 10k,1,1.0,37.1670
11,318,2.106964e+06,6088.773991,25.244980,55.304720,Sector 3,AL KARAMA,الكرامة,69095.0,70558.0,75066.0,35627.560131,6.389070,2 < x <= 20,10k < x <= 50k,1,1.0,75.0660
12,931,1.126888e+08,41997.295364,24.918575,55.444700,Sector 9,AL LESAILY,الليسيلي,2514.0,2633.0,2856.0,25.344132,8.469427,2 < x <= 20,10 < x <1000,1,1.0,2.8560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,376,6.411368e+06,10209.099291,25.100854,55.214970,Sector 3,AL BAESHAA SECOND,البرشاء الثانية,12800.0,14815.0,15523.0,2421.168133,4.778940,2 < x <= 20,1000 < x <= 10k,1,1.0,15.5230
216,631,3.867720e+07,35079.362818,25.098246,55.268770,Sector 6,HADAEQ SHEIKH MOHAMMED BIN RASHID,حدائق الشيخ محمد بن راشد,51.0,68.0,1563.0,40.411406,2198.529412,> 100,10 < x <1000,1,2.0,0.7815
220,364,4.825398e+06,8843.460063,25.144381,55.232024,Sector 3,AL GOZE IND. FIRST,القوز الصناعية الأولى,26569.0,26569.0,26075.0,5403.698834,-1.859310,-2 < x <=2,1000 < x <= 10k,1,1.0,26.0750
221,332,9.960425e+06,15279.078410,25.229747,55.253889,Sector 3,JUMEIRA FIRST,جميرا الأولى,17083.0,18843.0,20356.0,2043.687962,8.029507,2 < x <= 20,1000 < x <= 10k,1,1.0,20.3560


In [32]:
# # ##### Continuous Scale ####

community_pop_hcf_c = community_pop_hcf_e
#community_pop_hcf_c['HealthcareFacilities'] = community_pop_hcf_c['HealthcareFacilities'].fillna(0)
#community_pop_hcf_c.loc[community_pop_hcf_c['HealthcareFacilities'] == 21, 'HealthcareFacilities'] = np.nan # Removing Healthcare city
community_pop_hcf_c

fig = px.choropleth_mapbox(community_pop_hcf_c, 
                           geojson=dxb_communities_GJson, 
                           locations='COMM_NUM',
                           color=community_pop_hcf_c['Pop Per HealthCF'],
                           color_continuous_scale="RdYlGn_r",
                           range_color=(0, int(community_pop_hcf_c['Pop Per HealthCF'].max())),
                           featureidkey="properties.COMM_NUM",
                           hover_name='CNAME_E',
                           hover_data={'PopDensitybinned': False, 'HealthcareFacilities': True, 'Pop2019': True, 'Pop Per HealthCF':True},
                           mapbox_style='open-street-map',
                           zoom=9,
                           center={'lat': 25, 'lon': 55},
                           opacity=0.7,
                           width = 1200,
                           height=800,
                          )

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [33]:
community_pop_hcf_e.sum()

COMM_NUM                                                            26107
SHAPE_AREA                                                    8.24626e+08
SHAPE_LEN                                                          836762
lat                                                               1534.44
lon                                                               3374.08
Sector                  Sector 3Sector 3Sector 2Sector 3Sector 9Sector...
CNAME_E                 AL BADAAL SAFFA FIRSTAL RASHIDIYAAL KARAMAAL L...
CNAME_A                 البدعالصفا الأولىالراشديةالكرامةالليسيليالبراح...
Pop2017                                                       1.44527e+06
Pop2018                                                       1.51706e+06
Pop2019                                                       1.59806e+06
Pop2019density                                                     456490
PopGrowth19by18                                                   2605.95
PopGrowthbinned         2 < x <= 202 <


### There are 21 (132-111) Healthcare facilities in one community (No. 315) which has been excluded in the map